In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
environment = 'CartPole-v1'
env = gym.make(environment)

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info, _ = env.step(action)
        score += reward
    print(f'ep : {episode}, score : {score}')
env.close()

ep : 1, score : 19.0
ep : 2, score : 10.0
ep : 3, score : 10.0
ep : 4, score : 22.0
ep : 5, score : 11.0


In [5]:
log_path = os.path.join('Training_rein', 'Logs')

In [6]:
env = gym.make(environment)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, device='cuda')

Using cuda device


In [7]:
from timeit import default_timer as timer
start = timer()
model.learn(total_timesteps=10000)
env.close()
end = timer()
end - start

Logging to Training_rein/Logs/PPO_9
-----------------------------
| time/              |      |
|    fps             | 1452 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1249         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.008028535  |
|    clip_fraction        | 0.0795       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | -0.010069251 |
|    learning_rate        | 0.0003       |
|    loss                 | 6.95         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0116      |
|    value_loss           | 52.6         |
-----------------------

9.182181162999768

In [8]:
path = "ModelSave/model01"
#model.save(path)

In [9]:
#del model

In [10]:
model = PPO.load(path, env=env)
model

In [11]:
env = gym.make(environment, render_mode="human")

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

/home/kongpop/anaconda3/envs/WorkEnv3.11.6/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [32]:
episodes = 3
for episode in range(1, episodes+1):
    obs, _ = env.reset() 
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs) # <- use model.predict() instead of action sample
        obs, reward, done, info, _ = env.step(action)
        score += reward
    print(f'ep : {episode}, score : {score}')
env.close()

ep : 1, score : 194.0
ep : 2, score : 193.0
ep : 3, score : 178.0


In [42]:
training_log_path = "Training_rein/Logs/PPO_8"
training_log_path

'Training_rein/Logs/PPO_8'

In [44]:
!tensorboard --logdir={training_log_path}

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.17.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [24]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [26]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1) # it mean reward is 200++:
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback, #call this fn every 10,000 step
                            eval_freq=10000, #check in every 10,000 step
                            best_model_save_path="ModelSave/model01upgrage", #if mean reward is 200++ : create new model 
                            verbose=1)

In [28]:
model = PPO('MlpPolicy', env, verbose=1)

Using cuda device


In [30]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 1703 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1315        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008887499 |
|    clip_fraction        | 0.117       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | 0.00287     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.67        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0172     |
|    value_loss           | 51.6        |
-----------------------------------------
----------------------------------

/home/kongpop/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=271.20 +/- 129.96
Episode length: 271.20 +/- 129.96
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 271         |
|    mean_reward          | 271         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.010067271 |
|    clip_fraction        | 0.0819      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.604      |
|    explained_variance   | 0.338       |
|    learning_rate        | 0.0003      |
|    loss                 | 25          |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0154     |
|    value_loss           | 60.6        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 271.20  is above the threshold 200


In [36]:
#use neural network
net_arch = dict(pi=[128,128,128,128], vf=[128,128,128,128])

In [38]:
model = PPO('MlpPolicy', env, verbose=1, policy_kwargs={'net_arch':net_arch})

Using cuda device


In [40]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 1643 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1297        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013130082 |
|    clip_fraction        | 0.206       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.00137    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.24        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0225     |
|    value_loss           | 20          |
-----------------------------------------
----------------------------------

/home/kongpop/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=409.80 +/- 83.72
Episode length: 409.80 +/- 83.72
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 410         |
|    mean_reward          | 410         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.008744314 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.572      |
|    explained_variance   | 0.503       |
|    learning_rate        | 0.0003      |
|    loss                 | 12.4        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0207     |
|    value_loss           | 42.8        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 409.80  is above the threshold 200
